# Group 7 Project

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import datetime
import tensorflow as tf

## Retrieving Data

In [2]:
df = pd.read_csv("train.csv")
store_df = pd.read_csv("store.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Processing

Remove any closed stores or stores with 0 sales:

In [3]:
df = df.drop(df[df.Open == 0].index)
df = df.drop(df[df.Sales == 0].index)

Convert for time series:

In [4]:
df['Date'] = df['Date'].astype('datetime64[ns]')

Merge the two dataframes:

In [5]:
df = pd.merge(df, store_df, how='inner', on='Store')
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      844338 non-null  int64         
 1   DayOfWeek                  844338 non-null  int64         
 2   Date                       844338 non-null  datetime64[ns]
 3   Sales                      844338 non-null  int64         
 4   Customers                  844338 non-null  int64         
 5   Open                       844338 non-null  int64         
 6   Promo                      844338 non-null  int64         
 7   StateHoliday               844338 non-null  object        
 8   SchoolHoliday              844338 non-null  int64         
 9   StoreType                  844338 non-null  object        
 10  Assortment                 844338 non-null  object        
 11  CompetitionDistance        842152 non-null  float64 

### Handling Missing Values

In [7]:
df.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2186
CompetitionOpenSinceMonth    268600
CompetitionOpenSinceYear     268600
Promo2                            0
Promo2SinceWeek              423292
Promo2SinceYear              423292
PromoInterval                423292
dtype: int64

In [8]:
#Since there are only 3 missing values replace with median
df['CompetitionDistance'].fillna(df['CompetitionDistance'].median(), inplace = True)

#fill the rest with 0
df.fillna(0, inplace = True)

## Feature Engineering

Replace '0' with na values:

In [9]:
df['StateHoliday'] = df.StateHoliday.replace([0, '0'], np.nan)
df.StateHoliday.unique()

array([nan, 'a', 'b', 'c'], dtype=object)

### Dummy Encoding

In [10]:
dummy_columns = ['StoreType', 'Assortment', 'StateHoliday']
df = pd.get_dummies(df, columns=dummy_columns)

In [11]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,...,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,1,5,2015-07-31,5263,555,1,1,1,1270.0,9.0,...,0,0,1,0,1,0,0,0,0,0
1,1,4,2015-07-30,5020,546,1,1,1,1270.0,9.0,...,0,0,1,0,1,0,0,0,0,0
2,1,3,2015-07-29,4782,523,1,1,1,1270.0,9.0,...,0,0,1,0,1,0,0,0,0,0
3,1,2,2015-07-28,5011,560,1,1,1,1270.0,9.0,...,0,0,1,0,1,0,0,0,0,0
4,1,1,2015-07-27,6102,612,1,1,1,1270.0,9.0,...,0,0,1,0,1,0,0,0,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      844338 non-null  int64         
 1   DayOfWeek                  844338 non-null  int64         
 2   Date                       844338 non-null  datetime64[ns]
 3   Sales                      844338 non-null  int64         
 4   Customers                  844338 non-null  int64         
 5   Open                       844338 non-null  int64         
 6   Promo                      844338 non-null  int64         
 7   SchoolHoliday              844338 non-null  int64         
 8   CompetitionDistance        844338 non-null  float64       
 9   CompetitionOpenSinceMonth  844338 non-null  float64       
 10  CompetitionOpenSinceYear   844338 non-null  float64       
 11  Promo2                     844338 non-null  int64   

### Working with Time

In [13]:
#Splitting the Date as Day, month and year and adding 3 new columns
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Weekofyear'] = df['Date'].dt.weekofyear

<ipython-input-13-d58d24e736d5>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Weekofyear'] = df['Date'].dt.weekofyear


In [14]:
#How long the competition has been open
df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)

In [15]:
#How long the promo has been running
df['PromoOpen'] = 12 * (df.Year - df.Promo2SinceYear) + (df.Weekofyear - df.Promo2SinceWeek) / 4.0

In [16]:
df = df.drop(columns=['Weekofyear','Promo2SinceWeek', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'CompetitionOpenSinceYear', 'Open', 'PromoInterval'])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Store                844338 non-null  int64         
 1   DayOfWeek            844338 non-null  int64         
 2   Date                 844338 non-null  datetime64[ns]
 3   Sales                844338 non-null  int64         
 4   Customers            844338 non-null  int64         
 5   Promo                844338 non-null  int64         
 6   SchoolHoliday        844338 non-null  int64         
 7   CompetitionDistance  844338 non-null  float64       
 8   Promo2               844338 non-null  int64         
 9   StoreType_a          844338 non-null  uint8         
 10  StoreType_b          844338 non-null  uint8         
 11  StoreType_c          844338 non-null  uint8         
 12  StoreType_d          844338 non-null  uint8         
 13  Assortment_a  

In [18]:
#replace competitiondistance with furthest variable
df['CompetitionDistance'] = df.CompetitionDistance.replace(np.nan, df['CompetitionDistance'].max())
                                                           
#Competittion Open - use mean
df['CompetitionOpen'] = df.CompetitionOpen.replace(np.nan, df['CompetitionOpen'].max())

#PromoOpen - recode missing as 0
df['PromoOpen'] = df.PromoOpen.replace(np.nan, 0)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844338 entries, 0 to 844337
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Store                844338 non-null  int64         
 1   DayOfWeek            844338 non-null  int64         
 2   Date                 844338 non-null  datetime64[ns]
 3   Sales                844338 non-null  int64         
 4   Customers            844338 non-null  int64         
 5   Promo                844338 non-null  int64         
 6   SchoolHoliday        844338 non-null  int64         
 7   CompetitionDistance  844338 non-null  float64       
 8   Promo2               844338 non-null  int64         
 9   StoreType_a          844338 non-null  uint8         
 10  StoreType_b          844338 non-null  uint8         
 11  StoreType_c          844338 non-null  uint8         
 12  StoreType_d          844338 non-null  uint8         
 13  Assortment_a  

## Removing Outliers

In [20]:
for column in df.columns:
    print(column, df[column].dtypes)

Store int64
DayOfWeek int64
Date datetime64[ns]
Sales int64
Customers int64
Promo int64
SchoolHoliday int64
CompetitionDistance float64
Promo2 int64
StoreType_a uint8
StoreType_b uint8
StoreType_c uint8
StoreType_d uint8
Assortment_a uint8
Assortment_b uint8
Assortment_c uint8
StateHoliday_a uint8
StateHoliday_b uint8
StateHoliday_c uint8
Day int64
Month int64
Year int64
CompetitionOpen float64
PromoOpen float64


In [21]:
from scipy.stats import zscore

z_scores = np.abs(zscore(df.Sales))
z_scores

array([0.54544483, 0.62373562, 0.70041548, ..., 0.88728236, 0.7654967 ,
       0.6056933 ])

In [22]:
removed_df = df[z_scores < 3]
print('Removed data', len(df) - len(removed_df))
df = removed_df

Removed data 13443


## Splitting Data

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 830895 entries, 0 to 844337
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Store                830895 non-null  int64         
 1   DayOfWeek            830895 non-null  int64         
 2   Date                 830895 non-null  datetime64[ns]
 3   Sales                830895 non-null  int64         
 4   Customers            830895 non-null  int64         
 5   Promo                830895 non-null  int64         
 6   SchoolHoliday        830895 non-null  int64         
 7   CompetitionDistance  830895 non-null  float64       
 8   Promo2               830895 non-null  int64         
 9   StoreType_a          830895 non-null  uint8         
 10  StoreType_b          830895 non-null  uint8         
 11  StoreType_c          830895 non-null  uint8         
 12  StoreType_d          830895 non-null  uint8         
 13  Assortment_a  

In [24]:
X = df.drop(columns=['Date','Sales','Customers','Store','StateHoliday_b','StateHoliday_c'])
Y = df['Sales']

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_X

array([[ 0.85442228,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       [ 0.27291095,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       [-0.30860038,  1.12330986,  2.04438985, ...,  1.50221827,
        -0.68066285,  1.00770216],
       ...,
       [ 0.85442228, -0.89022632,  2.04438985, ..., -1.07021472,
        -0.6841365 ,  1.00509311],
       [ 0.27291095, -0.89022632,  2.04438985, ..., -1.07021472,
        -0.6841365 ,  1.00509311],
       [-0.30860038, -0.89022632,  2.04438985, ..., -1.07021472,
        -0.6841365 ,  1.00509311]])

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_rest, y_train, y_rest = train_test_split(scaled_X, Y, test_size=0.30)
x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5)

In [27]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 830895 entries, 0 to 844337
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   DayOfWeek            830895 non-null  int64  
 1   Promo                830895 non-null  int64  
 2   SchoolHoliday        830895 non-null  int64  
 3   CompetitionDistance  830895 non-null  float64
 4   Promo2               830895 non-null  int64  
 5   StoreType_a          830895 non-null  uint8  
 6   StoreType_b          830895 non-null  uint8  
 7   StoreType_c          830895 non-null  uint8  
 8   StoreType_d          830895 non-null  uint8  
 9   Assortment_a         830895 non-null  uint8  
 10  Assortment_b         830895 non-null  uint8  
 11  Assortment_c         830895 non-null  uint8  
 12  StateHoliday_a       830895 non-null  uint8  
 13  StateHoliday_b       830895 non-null  uint8  
 14  StateHoliday_c       830895 non-null  uint8  
 15  Day              

#### Store the data for the models

In [28]:
from numpy import savetxt, save

# Save as txt if needed in other csv format
# savetxt('csvs/x_train.csv', x_train, delimiter=',')
# savetxt('csvs/y_train.csv', y_train, delimiter=',')
# savetxt('csvs/x_test.csv', x_test, delimiter=',')
# savetxt('csvs/y_test.csv', y_test, delimiter=',')
# savetxt('csvs/x_val.csv', x_test, delimiter=',')
# savetxt('csvs/y_val.csv', y_test, delimiter=',')

# Save as binary for fastest and more efficient saving
save('npys/x_train.npy', x_train)
save('npys/y_train.npy', y_train)
save('npys/x_test.npy', x_test)
save('npys/y_test.npy', y_test)
save('npys/x_val.npy', x_val)
save('npys/y_val.npy', y_val)